In [39]:
import tensorflow as tf
import numpy as np
import scipy.io as sio
import os
from Hyperspectral.patchsize import  patch_size
from Hyperspectral.trian_models import CNN
from tensorflow.keras import optimizers

In [29]:
train_files=8
test_files=6
batch_size=30

**1. 加载数据**

In [33]:
def load_data():
    data_path = os.path.join("E:\\pythonProject\\DeepLearning", r'resources\data')
    train_data=np.array([])
    train_label=np.array([])
    test_data=np.array([])
    test_label=np.array([])

    for i in range(train_files):
        data_set=sio.loadmat(os.path.join(data_path,"train_"+str(patch_size)+
                                          '_'+str(i+1)+'.mat'))

        if i==0:
            train_data=data_set['train_data']
            train_label=data_set['train_label']
        else:
            train_data=np.concatenate((train_data,data_set['train_data']),
                                       axis=0)
            train_label=np.concatenate((train_label, data_set['train_label']),
                                        axis=1)


    for i in range(test_files):
        data_set=sio.loadmat(os.path.join(data_path,"test_"+str(patch_size)+
                                          '_'+str(i+1)+'.mat'))

        if i==0:
            test_data=data_set['test_data']
            test_label=data_set['test_label']
        else:
            test_data=np.concatenate((test_data,data_set['test_data']),
                                     axis=0)
            test_label=np.concatenate((test_label,data_set['test_label']),
                                      axis=1)

    train_data=np.transpose(train_data,(0,2,3,1))
    train_label=np.squeeze(np.transpose(train_label))
    test_data=np.transpose(test_data,(0,2,3,1))
    test_label=np.squeeze(np.transpose(test_label))

    train_db=tf.data.Dataset.from_tensor_slices((train_data,train_label))
    train_db=train_db.shuffle(1000).batch(batch_size)
    test_db=tf.data.Dataset.from_tensor_slices((test_data,test_label))
    test_db=test_db.shuffle(1000).batch(batch_size)

    return train_db,test_db

(3200, 220, 11, 11) (1, 3200)
(2400, 220, 11, 11) (1, 2400)


**2. 训练模型**

In [41]:
def train():
    kernal_size=3
    channel1=500
    channel2=100

    network=CNN(channel1,channel2,kernal_size)
    network.build(input_shape=(None,11,11,220))
    network.summary()

    #%%
    lr=0.01
    loss_step=0
    acc_step=0
    optimizer=optimizers.Adam(lr)

    total_acc=0
    total_num=0

    #%%
    for epoch in range(1000):
        for step,(x,y) in enumerate(train_db):
            loss_step+=1
            y=tf.one_hot(y,depth=16)
            with tf.GradientTape() as tape:
                logit = network(x)
                logit = tf.nn.softmax(logit,axis=1)
                loss = tf.losses.categorical_crossentropy()
                loss=tf.reduce_mean(loss,axis=10)

            grads=tape.gradient(loss,network.trainable_variables)
            optimizer.apply_gradients(zip(grads,network.trainable_variables))

        for step,(x,y) in enumerate(test_db):
            acc_step+=1
            prob=network(x)
            prob=tf.nn.softmax(prob,axis=1)
            pre=tf.argmax(prob,axis=1)
            correct=tf.equal


Model: "cnn_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            multiple                  990500    
_________________________________________________________________
re_lu_2 (ReLU)               multiple                  0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 multiple                  0         
_________________________________________________________________
batch_normalization_2 (Batch multiple                  2000      
_________________________________________________________________
conv2d_3 (Conv2D)            multiple                  450100    
_________________________________________________________________
re_lu_3 (ReLU)               multiple                  0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 multiple                  0     